***
## Метод get(): получение одного объекта

Чтобы получить отдельную запись через SQL-запрос, обычно обращаются к её первичному ключу. Например, получить запись, у которой id равен единице, можно таким запросом:

```sql
SELECT ...
FROM ...
WHERE id = 1; 
```

В проекте «Анфиса для друзей» в приложении *ice_cream* view-функция `ice_cream_detail()` должна принимать на вход `pk` мороженого, а в результате должна передать в шаблон информацию о мороженом с запрошенным `pk`.

Вместо метода `.filter()` применим метод `.get()`:

```py
# ice_cream/views.py
from django.shortcuts import render

from ice_cream.models import IceCream

def ice_cream_detail(request, pk):
    template_name = 'ice_cream/detail.html'
    # Вызываем .get() и в его параметрах указываем условия фильтрации:
    ice_cream = IceCream.objects.get(pk=pk)
    context = {
        'ice_cream': ice_cream,
    }
    return render(request, template_name, context)

...
```

Вызов `IceCream.objects.get(pk=pk)` вернёт не QuerySet, а отдельный объект модели IceCream с `pk`, полученным из запроса.

***
## Что за магия в запросе?

**Почему в коде Python — `pk`, а в SQL — `id`?**

Для обращения к первичным ключам моделей Django ORM можно применить «псевдоним» — `pk` (*primary key* — первичный ключ). Первичный ключ в таблице БД может называться как угодно — `id`, `unique_id` или `identifier`; но если в Django ORM обратиться к имени `pk` — фреймворк поймёт, что нужен первичный ключ таблицы, и в запросе укажет реальное имя поля.

**Откуда в SQL взялся LIMIT?**

В SQL-запросе, откуда ни возьмись, появилось странное ограничение: `LIMIT 21` («вернуть не более 21 записи»). Такое ограничение Django ORM автоматически добавляет ко всем запросам, которые по задумке должны возвращать единичный объект (это относится и к `.get()`). 

В аргументы метода `.get()` можно передавать любые поля с модификаторами, как и в метод `.filter()`. И если в базе найдётся несколько записей, соответствующих запросу, — возникнет ошибка: ведь ожидается, что метод `.get()` вернёт один-единственный объект.

Если запрос, предназначенный для получения одного объекта, вернёт более одной записи — это явная ошибка, а значит, нет смысла получать из базы все записи, соответствующие запросу. Чтобы понапрасну не тратить ресурсы — в Django ORM выставлено ограничение, `LIMIT 21`.



Попробуем вызвать ошибку: 

```py
# ice_cream/views.py
...

def ice_cream_detail(request, pk):
    ...
    ice_cream = IceCream.objects.get(title__contains='мороженое')
    ... 
```

Из базы, с которой мы работаем, вернётся семь записей:

![alt text](https://pictures.s3.yandex.net/resources/image_1736083609.png)

В сообщении об ошибке сказано: «метод `.get()` вернул более одного объекта: их целых семь!». Если же таких объектов будет более двадцати одного — Django ORM не будет вдаваться в подробности, найдено ли сто или тысяча объектов, а просто скажет «…их больше 21, перебор!».

***
## Функция get_object_or_404()

Если методом `.get()` запросить из базы несуществующий объект — Django выбросит исключение `DoesNotExist: IceCream matching query does not exist`. 

Запросите страницу http://127.0.0.1:8000/ice_cream/13/ в проекте, развёрнутом на вашем компьютере, — и эта ошибка появится в консоли и на странице.

Проект ведёт себя логично, но при запросе несуществующего объекта было бы лучше возвращать страницу с ошибкой 404 («запрошенной страницы не существует»). Пользователь погорюет немного и пойдёт смотреть сайт дальше.

Эта задача довольно востребована, и авторы Django создали встроенную функцию-шорткат `get_object_or_404()`, которая возвращает запрошенный объект, а если объект не существует — отправляет пользователю ошибку 404.

На вход функция `get_object_or_404()` ожидает

* аргумент `klass`: имя модели или QuerySet, из которого нужно получить запрошенный объект; название аргумента написали через k, потому что в Python слово `class` — зарезервированное, и в названиях переменных это слово применять нельзя;

* аргументы `*args, **kwargs` — параметры для фильтрации.

Перед применением `функции get_object_or_404()` необходимо её импортировать.

```py
from django.shortcuts import get_object_or_404, render

from ice_cream.models import IceCream

def ice_cream_detail(request, pk):
    template_name = 'ice_cream/detail.html'
    # Отфильтруй объект модели IceCream, 
    # у которого pk равен значению переменной из пути.
    # Если такого объекта не существует - верни 404 ошибку:
    ice_cream = get_object_or_404(IceCream, pk=pk)
    context = {
        'ice_cream': ice_cream,
    }
    return render(request, template_name, context) 
```

***
## Как устроена функция get_object_or_404()

`get_object_or_404()` — это короткая запись примерно такой конструкции:

```py
# Импортируем код состояния 404 ошибки.
from django.http import Http404
from django.shortcuts import render

from ice_cream.models import IceCream

def ice_cream_detail(request, pk):
    template = 'ice_cream/detail.html'
    # Готовимся перехватить исключение. 
    try:
        # Пытаемся получить объект с заданным pk:
          ice_cream = IceCream.objects.get(pk=pk)
    # Если объект не найден...
    except IceCream.DoesNotExist:
        # ...выбрасываем исключение Http404
        raise Http404('Такого мороженого не существует.')
        # Специальный обработчик перехватит выброшенное исключение
        # и среагирует установленным образом; 
        # по умолчанию - вернёт пользователю стандартную страницу ошибки 404.
    
    context = {
        'ice_cream': ice_cream,
    }
    return render(request, template, context)
```

***
В `get_object_or_404()` первым аргументом можно передать не только имя модели, но и QuerySet, этим можно воспользоваться для предварительной фильтрации объектов и для ограничения списка полей, которые вернутся в ответе на запрос. 

На примере будет нагляднее. В шаблоне страницы, описывающей отдельный сорт мороженого, нужны только поля `title` и `description`; получение данных для этой страницы может выглядеть так:

```py
from django.shortcuts import get_object_or_404, render

from ice_cream.models import IceCream

def ice_cream_detail(request, pk):
    template_name = 'ice_cream/detail.html'
    # Из модели IceCream получаем QuerySet, содержащий только
    # поля 'title' и 'description' всех записей.
    # Из этого QuerySet получаем запись, 
    # у которой значение поля pk равно значению переменной pk, 
    # полученной в аргументе view-функции.
    # Если объекта с указанным pk не существует - вернётся страница с ошибкой 404:
    ice_cream = get_object_or_404(
        # Первый аргумент - QuerySet:
        IceCream.objects.values('title', 'description'),
        # Второй аргумент - условие, по которому фильтруются записи из QuerySet:
        pk=pk
    )
    context = {
        'ice_cream': ice_cream,
    }
    return render(request, template_name, context)

... 
```

В результате запрос существенно сократится:

```sql
SELECT "ice_cream_icecream"."title",
       "ice_cream_icecream"."description"
FROM "ice_cream_icecream"
WHERE "ice_cream_icecream"."id" = "1"
LIMIT 21 
```

При получении списка объектов можно применить похожую функцию — `get_list_or_404()`. Логика работы этой функции проста: если по запросу получен список, в котором есть хоть один объект, — возвращаем этот список; если список пуст — выбрасываем исключение Http404. Исключение, как и в случае с `get_object_or_404()`, будет перехвачено и обработано.

***
## Получить первый или последний объект из QuerySet: методы .first() и .last()

Ещё один способ вернуть отдельный объект — получить QuerySet и методом `.first()` или  `.last()` получить из него, соответственно, первый или последний объект. 

```py
# Такой вызов вернёт первый элемент из QuerySet
IceCream.objects.filter(is_published=True).order_by('pk').first() 
```

Методы `.first()` и `.last()` нужно применять только к отсортированным QuerySet. Если последовательность не отсортирована — результат может быть неожиданным: без сортировки сложно угадать, какой объект будет первым в наборе, а какой — последним.

***
## Хозяйке на заметку

* [Документация по QuerySet](https://docs.djangoproject.com/en/3.2/ref/models/querysets/#django.db.models.query.QuerySet)

* [Документация функции get_object_or_404()](https://docs.djangoproject.com/en/3.2/topics/http/shortcuts/#get-object-or-404)

* [Документация метода get_list_or_404()](https://docs.djangoproject.com/en/3.2/topics/http/shortcuts/#get-list-or-404)

* [Документация методов .first() и .last()](https://docs.djangoproject.com/en/3.2/ref/models/querysets/#first)